## GENRE PREDICTION USING LLM

In [1]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 4.0 MB/s eta 0:00:00


In [10]:
from langchain_groq import ChatGroq
import os
import pandas as pd
import time

os.environ["GROQ_API_KEY"] = "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"

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=100,
    max_retries=2,
)

def predict_genres(movie_name, movie_summary):
    try:
        print(f"Processing: {movie_name}...")

        genres_list = [
            "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
            "Film Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
        ]

        messages = [
            (
                "system",
                "You are an expert in movie genre classification. Your task is to classify the movie summary into relevant genres based on the predefined list of genres. "
                "The predefined genres are: " + ", ".join(genres_list) + ". "
                "Only return the list of genres in Python list format, without any additional text."
            ),
            (
                "human",
                f"""Given the following movie name and summary, return the genres as a Python list:
                Movie Name: {movie_name}
                Summary: {movie_summary}
                Genres: """
            )
        ]

        ai_msg = llm.invoke(messages)
        return ai_msg.content.strip()

    except Exception as e:
        print(f"Error predicting genres: {e}")
        return "[]"  # Return an empty list if an error occurs


In [32]:
movies = pd.read_csv("movies_sub.csv") 

In [12]:
genres_list = [
            "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
            "Film Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
        ]

In [ ]:
unprocessed_movies = movies[movies["Processed"] == False]
len(unprocessed_movies)


In [37]:
batch_size = 10
for i in range(0, len(unprocessed_movies), batch_size):
    batch = unprocessed_movies[i : i + batch_size]

    # Predict genres once per movie
    batch["Predicted Genres"] = batch.apply(lambda row: predict_genres(row["Movie_Name"], row["Llama Summary Few Shot"]), axis=1)

    # Convert predicted genres to one-hot encoding
    for genre in genres_list:
        batch[genre] = batch["Predicted Genres"].apply(lambda genres: 1 if genre in genres else 0)

    # Update the original DataFrame and save checkpoint
    movies.loc[batch.index, genres_list] = batch[genres_list]
    movies.loc[batch.index, "Processed"] = batch[genres_list].sum(axis=1) > 0  # Mark as True if any genre is assigned
    movies.to_csv("movies_sub.csv", index=False)

    print(f"Checkpoint saved. Processed {i + batch_size} movies.")
    time.sleep(1)  # Optional: Add a small delay between batches

print("Processing completed!")

Processing: Empire Strikes Back, The (1980)...
Processing: Crude Oasis, The (1995)...
Processing: Story of Xinghua, The (1993)...
Processing: Lashou shentan (1992)...


<ipython-input-37-5fd86d1bbfb0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["Predicted Genres"] = batch.apply(lambda row: predict_genres(row["Movie_Name"], row["Llama Summary Few Shot"]), axis=1)
<ipython-input-37-5fd86d1bbfb0>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch[genre] = batch["Predicted Genres"].apply(lambda genres: 1 if genre in genres else 0)


Checkpoint saved. Processed 10 movies.
Processing completed!
